# Gewinnwahrscheinlichkeiten - exakt und gerundet
Mit dem folgenden Programm werden die Wahrscheinlichkeiten $P_A(V,W)$, $P_B(V,W)$ und $P_U(V,W)$ berechnet.

#### Eingabe

Die Wahrscheinlichkeiten können als Brüche eingegeben werden.

**Beispiel**: Für $p=(1/2,1/3,1/6)$ lautet die Eingabe

*p = (Fraction(1, 2), Fraction(1, 3), Fraction(1, 6))*.

A hat die Strategie $V=(3,2,1)$ und B hat $W=(4,2,0)$ gewählt.

Die Anzahl an Feldern und an Chips kann verändert werden.

### Mathematischer Hintergrund ###

Mithilfe des folgenden Python-Programms können die Wahrscheinlichkeiten berechnet werden. Es gelten die folgenden Abbruchkriterien:

$$
\begin{eqnarray}
\text{P}_A(V,W) &  =  &  1, \ \text{falls} \  V = \mathbf{0}, \ W \neq \mathbf{0},\\ 
\text{P}_A(V,W)   &  =  &  0, \ \text{falls} \  V \neq \mathbf{0}, \ W = \mathbf{0},\\ 
\text{P}_A(V,W)  &  =  &   0, \ \text{falls} \  V=W.
\end{eqnarray}
$$

$V_j$ und $W_j$ bezeichnen die aus $V$ bzw. $W$ durch Entfernen eines Chips von Feld $j$ entstehenden  Chip-Verteilungen.

Wird $\mathbb{1}(\cdot) := 1$ bzw. $\mathbb{1}(\cdot) := 0$ gesetzt je nachdem, ob eine anstelle des Punktes stehende Aussage zutrifft oder nicht, so ergbt sich - zusammen mit 
$$
\begin{equation}
s:= 1- \sum_{j=1}^m p_j \cdot \mathbb{1}(v_j=0,w_j =0)
\end{equation}
$$
die Rekursionsformel

$$ 
\begin{align*}
  \text{P}_A(V,W)
  &  =  \sum\limits_{j=1}^m \frac{p_j}{s} \cdot \text{P}_A(V_j,W_j) \cdot  \mathbb{1}(v_j \geq 1,w_j \geq 1) \\
  & \quad  + \sum\limits_{j=1}^m \frac{p_j}{s} \cdot \text{P}_A(V,W_j) \cdot  \mathbb{1}(v_j = 0,w_j \geq 1)\\
  & \quad  + \sum\limits_{j=1}^m \frac{p_j}{s} \cdot \text{P}_A(V_j,W) \cdot  \mathbb{1}(v_j \geq 1,w_j = 0). 
\end{align*} 
$$

Ein rekursiv arbeitendes Programm benötigt nur diese Rekursionsformel, in der man mit
$V = S$ und $W=T$ startet, sowie die Abbruchbedingungen. Für  $\text{P}_B(V,W)$ gilt entsprechendes.

In [6]:
from functools import lru_cache
from fractions import Fraction

# Eingabe
p = (Fraction(1, 2), Fraction(1, 3), Fraction(1, 6))
V = (3, 2, 1)  # Chips von A
W = (4, 2, 0)  # Chips von B

@lru_cache(None)
def P_A(V, W, p):
    m = len(V)
    
    # Abbruchbedingungen mit Vektoren
    if V == W:
        return (Fraction(0), Fraction(0), Fraction(1))  # Unentschieden
    if all(x == 0 for x in V):
        return (Fraction(1), Fraction(0), Fraction(0))  # A gewinnt
    if all(x == 0 for x in W):
        return (Fraction(0), Fraction(1), Fraction(0))  # B gewinnt
    
    # s-Wert berechnen
    s = Fraction(1) - sum(Fraction(p[j]) for j in range(m) if V[j] == 0 and W[j] == 0)
    
    # Rekursion
    P_A_total = Fraction(0)
    P_B_total = Fraction(0)
    P_U_total = Fraction(0)
    
    for j in range(m):
        if V[j] > 0 or W[j] > 0:
            Vj = tuple(v - 1 if i == j and v > 0 else v for i, v in enumerate(V))
            Wj = tuple(w - 1 if i == j and w > 0 else w for i, w in enumerate(W))
            P_A_next, P_B_next, P_U_next = P_A(Vj, Wj, p)
            P_A_total += (Fraction(p[j]) / s) * P_A_next
            P_B_total += (Fraction(p[j]) / s) * P_B_next
            P_U_total += (Fraction(p[j]) / s) * P_U_next
    
    return (P_A_total, P_B_total, P_U_total)


# Berechnung
P_A_value, P_B_value, P_U_value = P_A(V, W, p)
 
# Ausgabe
print("Ausgabe der Wahrscheinlichkeiten, exakt und auf 4 Nachkommastellen")
print()
print(f"Gewinnwahrscheinlichkeit für A:")
print(f"P_A({V},{W})={P_A_value}")
print(f"P_A({V},{W})={float(P_A_value):.4}")
print()
print(f"Gewinnwahrscheinlichkeit für B:")
print(f"P_B({V},{W})={P_B_value}")
print(f"P_B({V},{W})={float(P_B_value):.4}")
print()
print(f"Wahrscheinlichkeit für unentschieden:")
print(f"P_U({V},{W})={P_U_value}")
print(f"P_U({V},{W})={float(P_U_value):.4}")

Ausgabe der Wahrscheinlichkeiten, exakt und auf 4 Nachkommastellen

Gewinnwahrscheinlichkeit für A:
P_A((3, 2, 1),(4, 2, 0))=1181921/2400000
P_A((3, 2, 1),(4, 2, 0))=0.4925

Gewinnwahrscheinlichkeit für B:
P_B((3, 2, 1),(4, 2, 0))=65/256
P_B((3, 2, 1),(4, 2, 0))=0.2539

Wahrscheinlichkeit für unentschieden:
P_U((3, 2, 1),(4, 2, 0))=9511/37500
P_U((3, 2, 1),(4, 2, 0))=0.2536
